In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


In [ ]:
! pip install wandb

In [ ]:
! pip install sacremoses

In [ ]:
import torch 
torch.cuda.is_available()
print(torch.version.cuda)

In [ ]:
! git clone https://github.com/pytorch/fairseq

In [ ]:
%cd fairseq
! pip install --editable ./
%cd ..

In [ ]:
import wandb
wandb.login()

In [ ]:
! fairseq-preprocess --source-lang ar --target-lang en \
  --trainpref data_non_processed/train \
  --validpref data_non_processed/valid \
  --testpref  data_non_processed/test \
  --destdir data-bin/data.tokenized.ar-en \
  --thresholdsrc 2 \
  --thresholdtgt 2

In [ ]:
!pip install tensorboardX

In [ ]:
! fairseq-train data-bin/data.tokenized.ar-en \
  --arch transformer \
  --dropout 0.1 \
  --max-tokens 256 \
  --attention-dropout 0.1 \
  --activation-dropout 0.1 \
  --encoder-embed-dim 128 \
  --encoder-ffn-embed-dim 256 \
  --encoder-layers 3 \
  --encoder-attention-heads 8 \
  --encoder-learned-pos \
  --decoder-embed-dim 128 \
  --decoder-ffn-embed-dim 256 \
  --decoder-layers 3 \
  --decoder-attention-heads 8 \
  --decoder-learned-pos \
  --max-epoch 10 \
  --optimizer adam \
  --lr 5e-4 \
  --batch-size 128 \
  --seed 1 \
  --save-interval 2 \
  --memory-efficient-fp16 \
  --update-freq 1 \
  --save-dir checkpoints \
  --wandb-project "en to ar Translation project 2.0"


In [ ]:
!fairseq-interactive \
  --path trained_model/model.pt \
  --tokenizer moses \
  --task translation --cpu \
  --beam 5 \
  --sacrebleu --remove-bpe \
  --buffer-size 256 \
  --source-lang ar --target-lang en  trained_model/

In [ ]:
import torch
from fairseq.models.transformer import TransformerModel
from sacremoses import MosesTokenizer

# Load the Fairseq trained model
model_path = "trained_model"
model = TransformerModel.from_pretrained(
    model_path,
    checkpoint_file="model.pt",
    data_name_or_path=model_path,
    source_lang="ar",  # Specify the source language code
    target_lang="en",  # Specify the target language code
)

# Set the model to evaluation mode
model.eval()

# Create Moses tokenizer
tokenizer = MosesTokenizer(lang="ar")

# Input sentence in Arabic
input_sentence = "في الهندسة الكهربائية والعلوم الدائرة المكافئة تشير إلى دائرة نظرية تحتفظ بكل الخصائص الكهربائية لدائرة كهربية معطاة ويعتقد أن الدائرة المكافئة تبسط الحسابات وعلى نطاق أوسع فإنها الشكل الأبسط لدائرة أكثر تعقيدا للمساعدة في تحليلها في شكلها الشائع فإن الدائرة المكافئة تتكون من عناصر خطية وغير فعالة ولكن الدوائر المكافئة الأكثر تعقيدا تقوم بتقريب السلوك الغير خطي للدائرة الأصلية أيضا  وتسمى الدوائر الأكثر تعقيدا بالنماذج الشاملة للدائرة الأصلية وكمثال على النماذج الشاملة هي دائرة بويل لمضخم التشغيل"

# Tokenize the input sentence using Moses tokenizer
tokenized_sentence = tokenizer.tokenize(input_sentence)

# Convert the tokenized sentence to PyTorch tensor
tokens = [model.task.source_dictionary.index(token) for token in tokenized_sentence]
input_tensor = torch.LongTensor(tokens).unsqueeze(0)  # Add batch dimension

# Generate translation using the model
with torch.no_grad():
    translation = model.generate(input_tensor, beam=5)

# Get the translated sentence without special tokens
translation_tokens = translation[0][0]["tokens"]

# Remove consecutive duplicate tokens
unique_tokens = [translation_tokens[0]]
for i in range(1, len(translation_tokens)):
    if translation_tokens[i] != translation_tokens[i-1]:
        unique_tokens.append(translation_tokens[i])

# Convert the translation tokens to a list of strings
decoded_tokens = [model.task.target_dictionary[token.item()] for token in unique_tokens if token.item() != model.task.target_dictionary.eos()]

# Join the decoded tokens into a single string
translated_sentence = " ".join(decoded_tokens)

print("Input Sentence:", input_sentence)
print("Translated Sentence:", translated_sentence)












In [2]:
#interactive use
import torch
from fairseq.models.transformer import TransformerModel

# Load the Fairseq trained model
model_path = "trained_model"
model = TransformerModel.from_pretrained(
    model_path,
    checkpoint_file="fairseqmodel.pt",
    data_name_or_path=model_path,
    source_lang="ar",  # Specify the source language code
    target_lang="en",  # Specify the target language code
)

# Set the model to evaluation mode
model.eval()

# Translate interactively
while True:
    # Take user input
    input_sentence = input("Enter a sentence in Arabic (or 'q' to quit): ")

    if input_sentence.lower() == "q":
        break

    # Tokenize the input sentence
    tokens = model.encode(input_sentence)

    # Convert the tokens to PyTorch tensor
    input_tensor = torch.LongTensor(tokens).unsqueeze(0)  # Add batch dimension

    # Generate translation using the model
    with torch.no_grad():
        translation = model.generate(input_tensor, beam=5)

    # Get the translated sentence without special tokens
    translation_sentence = model.decode(translation[0][0]["tokens"])

    print("Translated Sentence:", translation_sentence)


2023-06-25 01:24:10 | INFO | fairseq.file_utils | loading archive file trained_model
2023-06-25 01:24:10 | INFO | fairseq.file_utils | loading archive file trained_model
2023-06-25 01:24:27 | INFO | fairseq.tasks.translation | [ar] dictionary: 363624 types
2023-06-25 01:24:27 | INFO | fairseq.tasks.translation | [en] dictionary: 260400 types
2023-06-25 01:24:35 | INFO | fairseq.models.fairseq_model | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': None, 'wandb_project': 'Translation 3.0 ar to en', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': True, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': No

Enter a sentence in Arabic (or 'q' to quit):  أحمد ذهب إلى المدرسة


C:\Users\asus\anaconda3\envs\myenv\lib\site-packages\fairseq\models\transformer\transformer_encoder.py:281: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\NestedTensorImpl.cpp:179.)
  x = torch._nested_tensor_from_mask(


Translated Sentence: ahmed went to school


Enter a sentence in Arabic (or 'q' to quit):  أهلا و سهلا


...

Enter a sentence in Arabic (or 'q' to quit):  رافع


Translated Sentence: the lifter


Enter a sentence in Arabic (or 'q' to quit):  موزة


Translated Sentence: banana


KeyboardInterrupt: Interrupted by user